# Test NIP-66 Relay Monitoring and Discovery

Interactive notebook for testing the `Nip66` model:
- Test relay connectivity (open, read, write) with RTT measurements
- Probe test results (success/failure with raw rejection reasons)
- SSL certificate inspection
- DNS resolution
- Network/ASN lookup
- Geolocation lookup (requires GeoIP databases)
- HTTP headers extraction
- Proxy support for Tor/I2P/Loki relays

In [ ]:
import sys
sys.path.insert(0, "../src")

import asyncio
import json
from nostr_sdk import EventBuilder, Filter, Keys, Kind
from models.nip66 import Nip66, Nip66TestError
from models.relay import Relay
from models.metadata import Metadata

print("Imports loaded successfully")

## 1. Setup Test Keys and Event Builder

In [ ]:
# Generate ephemeral keys for testing
keys = Keys.generate()
print(f"Test pubkey: {keys.public_key().to_hex()[:16]}...")

# Create test event builder and read filter
event_builder = EventBuilder.text_note("NIP-66 test event")
read_filter = Filter().kind(Kind(1)).limit(1)

print("Keys and filters ready")

## 2. Basic Test - DNS, SSL, HTTP Only (No GeoIP)

In [ ]:
# Test relay without RTT or Geo (fastest test)
relay = Relay("wss://relay.damus.io")
print(f"Relay: {relay.url}")
print(f"Network: {relay.network}")
print(f"Host: {relay.host}")

In [ ]:
# Run basic test (DNS, SSL, HTTP only)
nip66 = await Nip66.test(
    relay,
    timeout=10.0,
    run_rtt=False,  # Skip RTT to speed up
    run_geo=False,  # Skip geo (no GeoIP db)
    run_net=False,  # Skip net (no GeoIP db)
)

print(f"Test successful: {nip66 is not None}")
print(f"Type: {type(nip66).__name__}")

## 3. DNS Metadata

In [ ]:
# Access DNS metadata
if nip66.dns_metadata:
    dns = nip66.dns_metadata.data
    print("=== DNS Metadata ===")
    print(f"dns_ips: {dns.get('dns_ips')}")
    print(f"dns_ips_v6: {dns.get('dns_ips_v6')}")
    print(f"dns_cname: {dns.get('dns_cname')}")
    print(f"dns_ns: {dns.get('dns_ns')}")
    print(f"dns_reverse: {dns.get('dns_reverse')}")
    print(f"dns_ttl: {dns.get('dns_ttl')} seconds")
else:
    print("No DNS metadata available")

## 4. SSL Metadata

In [ ]:
# Access SSL metadata
if nip66.ssl_metadata:
    ssl_data = nip66.ssl_metadata.data
    print("=== SSL Metadata ===")
    print(f"ssl_valid: {ssl_data.get('ssl_valid')}")
    print(f"ssl_subject_cn: {ssl_data.get('ssl_subject_cn')}")
    print(f"ssl_issuer: {ssl_data.get('ssl_issuer')}")
    print(f"ssl_issuer_cn: {ssl_data.get('ssl_issuer_cn')}")
    print(f"ssl_protocol: {ssl_data.get('ssl_protocol')}")
    print(f"ssl_cipher: {ssl_data.get('ssl_cipher')}")
    print(f"ssl_cipher_bits: {ssl_data.get('ssl_cipher_bits')}")
    print(f"ssl_san: {ssl_data.get('ssl_san')}")
else:
    print("No SSL metadata available")

In [ ]:
# Full SSL details
if nip66.ssl_metadata:
    from datetime import datetime
    ssl_data = nip66.ssl_metadata.data
    print("=== SSL Certificate Details ===")
    
    expires = ssl_data.get('ssl_expires')
    not_before = ssl_data.get('ssl_not_before')
    if expires:
        print(f"Expires: {datetime.fromtimestamp(expires)}")
    if not_before:
        print(f"Not Before: {datetime.fromtimestamp(not_before)}")
    
    print(f"Serial: {ssl_data.get('ssl_serial')}")
    print(f"Version: {ssl_data.get('ssl_version')}")
    print(f"Fingerprint: {ssl_data.get('ssl_fingerprint')}")

## 5. HTTP Metadata

In [ ]:
# Access HTTP metadata
if nip66.http_metadata:
    http = nip66.http_metadata.data
    print("=== HTTP Metadata ===")
    print(f"http_server: {http.get('http_server')}")
    print(f"http_powered_by: {http.get('http_powered_by')}")
else:
    print("No HTTP metadata available")

## 6. Full Test with RTT and Probe (Open/Read/Write)

In [ ]:
# Full test including RTT (slower, requires WebSocket connection)
relay_full = Relay("wss://relay.damus.io")

nip66_full = await Nip66.test(
    relay_full,
    timeout=15.0,
    keys=keys,
    event_builder=event_builder,
    read_filter=read_filter,
    run_rtt=True,
    run_ssl=True,
    run_dns=True,
    run_http=True,
    run_geo=False,  # Skip geo (no GeoIP db)
    run_net=False,  # Skip net (no GeoIP db)
)

print(f"Full test completed for {relay_full.url}")

In [ ]:
# Access RTT metadata
if nip66_full.rtt_metadata:
    rtt = nip66_full.rtt_metadata.data
    print("=== RTT Metadata ===")
    print(f"rtt_open: {rtt.get('rtt_open')}ms (connection time)")
    print(f"rtt_read: {rtt.get('rtt_read')}ms (first event received)")
    print(f"rtt_write: {rtt.get('rtt_write')}ms (event accepted)")
else:
    print("No RTT metadata available")

In [ ]:
# Access Probe metadata (success/failure with raw rejection reasons)
if nip66_full.probe_metadata:
    probe = nip66_full.probe_metadata.data
    print("=== Probe Metadata ===")
    print(f"probe_open_success: {probe.get('probe_open_success')}")
    print(f"probe_open_reason: {probe.get('probe_open_reason')}")
    print(f"probe_read_success: {probe.get('probe_read_success')}")
    print(f"probe_read_reason: {probe.get('probe_read_reason')}")
    print(f"probe_write_success: {probe.get('probe_write_success')}")
    print(f"probe_write_reason: {probe.get('probe_write_reason')}")
else:
    print("No Probe metadata available")

## 7. Test Multiple Relays

In [ ]:
# Test multiple relays in parallel
relay_urls = [
    "wss://relay.damus.io",
    "wss://nos.lol",
    "wss://relay.nostr.band",
    "wss://nostr.wine",
    "wss://relay.snort.social",
]

async def test_nip66_safe(url: str) -> tuple[str, Nip66 | None, Exception | None]:
    relay = Relay(url)
    try:
        result = await Nip66.test(
            relay,
            timeout=10.0,
            run_rtt=False,
            run_geo=False,
            run_net=False,
        )
        return url, result, None
    except Exception as e:
        return url, None, e

results = await asyncio.gather(*[test_nip66_safe(url) for url in relay_urls])

print("=== Multiple Relay Results ===")
for url, nip66_result, error in results:
    if nip66_result:
        ssl_valid = "N/A"
        if nip66_result.ssl_metadata:
            ssl_valid = nip66_result.ssl_metadata.data.get('ssl_valid', 'N/A')
        dns_ips = "N/A"
        if nip66_result.dns_metadata:
            ips = nip66_result.dns_metadata.data.get('dns_ips')
            dns_ips = ips[0] if ips else "N/A"
        print(f"✓ {url}: SSL={ssl_valid}, IP={dns_ips}")
    else:
        print(f"✗ {url}: {type(error).__name__}")

## 8. Conversion to RelayMetadata

In [ ]:
# Convert to RelayMetadata for DB storage
# Returns 7 RelayMetadata objects: (rtt, probe, ssl, geo, net, dns, http)
rtt_rm, probe_rm, ssl_rm, geo_rm, net_rm, dns_rm, http_rm = nip66_full.to_relay_metadata()

print("=== RelayMetadata Objects ===")
for name, rm in [
    ("RTT", rtt_rm),
    ("PROBE", probe_rm),
    ("SSL", ssl_rm),
    ("GEO", geo_rm),
    ("NET", net_rm),
    ("DNS", dns_rm),
    ("HTTP", http_rm),
]:
    if rm:
        print(f"{name}: type={rm.metadata_type}, generated_at={rm.generated_at}")
    else:
        print(f"{name}: None")

## 9. Geolocation and Network Test (Requires GeoIP Databases)

To test geolocation and network info, you need MaxMind GeoLite2 databases:
- GeoLite2-City.mmdb (for geo_metadata)
- GeoLite2-ASN.mmdb (for net_metadata)

Download from: https://dev.maxmind.com/geoip/geolite2-free-geolocation-data

In [ ]:
# Example: Test with GeoIP databases (uncomment to run)
# import geoip2.database
#
# city_reader = geoip2.database.Reader("/path/to/GeoLite2-City.mmdb")
# asn_reader = geoip2.database.Reader("/path/to/GeoLite2-ASN.mmdb")
#
# relay_geo = Relay("wss://relay.damus.io")
# nip66_geo = await Nip66.test(
#     relay_geo,
#     timeout=10.0,
#     city_reader=city_reader,
#     asn_reader=asn_reader,
#     run_rtt=False,
#     run_geo=True,
#     run_net=True,
# )
#
# if nip66_geo.geo_metadata:
#     geo = nip66_geo.geo_metadata.data
#     print("=== Geo Metadata ===")
#     print(f"geo_country: {geo.get('geo_country')} ({geo.get('geo_country_name')})")
#     print(f"geo_city: {geo.get('geo_city')}")
#     print(f"geo_region: {geo.get('geo_region')}")
#     print(f"geo_lat/lon: {geo.get('geo_lat')}, {geo.get('geo_lon')}")
#     print(f"geohash: {geo.get('geohash')}")
#     print(f"geo_tz: {geo.get('geo_tz')}")
#     print(f"geo_is_eu: {geo.get('geo_is_eu')}")
#
# if nip66_geo.net_metadata:
#     net = nip66_geo.net_metadata.data
#     print("\n=== Net Metadata ===")
#     print(f"net_ip: {net.get('net_ip')}")
#     print(f"net_ipv6: {net.get('net_ipv6')}")
#     print(f"net_asn: {net.get('net_asn')}")
#     print(f"net_asn_org: {net.get('net_asn_org')}")
#     print(f"net_network: {net.get('net_network')}")
#     print(f"net_network_v6: {net.get('net_network_v6')}")
#
# city_reader.close()
# asn_reader.close()

print("Geo/Net test skipped (uncomment to run with GeoIP databases)")

## 10. Tor Relay Test (Requires Proxy)

To test Tor relays, you need a running Tor proxy (SOCKS5 on port 9050).

In [ ]:
# Example: Test Tor relay (uncomment to run with local Tor proxy)
# tor_relay = Relay("ws://oxtrdevav64z64yb7x6rjg4ntzqjhedm5b5zjqulugknhzr46ny2qbad.onion")
# tor_proxy = "socks5://127.0.0.1:9050"
#
# try:
#     nip66_tor = await Nip66.test(
#         tor_relay,
#         timeout=30.0,
#         keys=keys,
#         event_builder=event_builder,
#         read_filter=read_filter,
#         proxy_url=tor_proxy,
#         run_rtt=True,
#         run_ssl=False,  # No SSL for .onion
#         run_dns=False,  # No DNS for .onion
#         run_geo=False,  # No geo for .onion
#         run_net=False,  # No net for .onion
#         run_http=True,
#     )
#     print(f"Tor relay test succeeded")
#     if nip66_tor.rtt_metadata:
#         print(f"RTT open: {nip66_tor.rtt_metadata.data.get('rtt_open')}ms")
#     if nip66_tor.probe_metadata:
#         print(f"Probe open success: {nip66_tor.probe_metadata.data.get('probe_open_success')}")
# except Nip66TestError as e:
#     print(f"Tor fetch failed: {e.cause}")

print("Tor test skipped (uncomment to run with local Tor proxy)")

## 11. Test with Synthetic Data

In [ ]:
# Create Nip66 from synthetic data
test_relay = Relay("wss://test.relay.example")

synthetic_rtt = Metadata({
    "rtt_open": 150,
    "rtt_read": 200,
    "rtt_write": 180,
})

synthetic_probe = Metadata({
    "probe_open_success": True,
    "probe_read_success": True,
    "probe_write_success": False,
    "probe_write_reason": "auth-required: please authenticate",
})

synthetic_ssl = Metadata({
    "ssl_valid": True,
    "ssl_issuer": "Let's Encrypt",
    "ssl_protocol": "TLSv1.3",
})

nip66_synthetic = Nip66(
    relay=test_relay,
    rtt_metadata=synthetic_rtt,
    probe_metadata=synthetic_probe,
    ssl_metadata=synthetic_ssl,
)

print("=== Synthetic Nip66 ===")
print(f"rtt_open: {nip66_synthetic.rtt_metadata.data.get('rtt_open')}")
print(f"probe_open_success: {nip66_synthetic.probe_metadata.data.get('probe_open_success')}")
print(f"probe_write_success: {nip66_synthetic.probe_metadata.data.get('probe_write_success')}")
print(f"probe_write_reason: {nip66_synthetic.probe_metadata.data.get('probe_write_reason')}")
print(f"ssl_valid: {nip66_synthetic.ssl_metadata.data.get('ssl_valid')}")

## 12. Error Handling

In [ ]:
# Test: Invalid relay URL
invalid_relay = Relay("wss://nonexistent.relay.invalid")

try:
    result = await Nip66.test(
        invalid_relay,
        timeout=3.0,
        run_rtt=False,
        run_geo=False,
        run_net=False,
    )
except Nip66TestError as e:
    print("=== Nip66TestError ===")
    print(f"relay: {e.relay.url}")
    print(f"cause: {type(e.cause).__name__}")

In [ ]:
# Test: Empty metadata raises ValueError
try:
    nip66_empty = Nip66(
        relay=test_relay,
        rtt_metadata=None,
        probe_metadata=None,
        ssl_metadata=None,
        geo_metadata=None,
        net_metadata=None,
        dns_metadata=None,
        http_metadata=None,
    )
except ValueError as e:
    print("=== Empty Metadata ===")
    print(f"Correctly raised: {e}")

## 13. Class Defaults

In [ ]:
print("=== Class Defaults ===")
print(f"_DEFAULT_TEST_TIMEOUT: {Nip66._DEFAULT_TEST_TIMEOUT} seconds")

## 14. Full JSON Output

In [ ]:
# Dump all metadata as JSON
print("=== Full Nip66 Data ===")
print(f"Relay: {nip66_full.relay.url}")
print(f"Generated at: {nip66_full.generated_at}")
print()

for name, metadata in [
    ("RTT", nip66_full.rtt_metadata),
    ("PROBE", nip66_full.probe_metadata),
    ("SSL", nip66_full.ssl_metadata),
    ("GEO", nip66_full.geo_metadata),
    ("NET", nip66_full.net_metadata),
    ("DNS", nip66_full.dns_metadata),
    ("HTTP", nip66_full.http_metadata),
]:
    if metadata:
        print(f"--- {name} ---")
        print(json.dumps(metadata.data, indent=2, default=str))
        print()

## Done!

### Key Points:
- `Nip66.test()` runs all enabled tests in parallel
- Access fields via `nip66.rtt_metadata.data["key"]`, etc.
- RTT test also collects `probe_metadata` with success/failure and raw rejection reasons
- RTT test requires `keys`, `event_builder`, and `read_filter`
- Geo test requires `city_reader` (GeoIP City database)
- Net test requires `asn_reader` (GeoIP ASN database)
- SSL/DNS tests only work for clearnet relays
- Use `proxy_url` for Tor/I2P/Loki relays (RTT and HTTP tests)
- Converts to up to 7 `RelayMetadata` objects for database storage:
  - `nip66_rtt` - Round-trip times
  - `nip66_probe` - Probe test results (success/failure)
  - `nip66_ssl` - SSL certificate data
  - `nip66_geo` - Geolocation data
  - `nip66_net` - Network/ASN data
  - `nip66_dns` - DNS resolution data
  - `nip66_http` - HTTP headers data